In [ ]:
## Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36
import os
import sys
import platform
import pandas as pd
import re
from datetime import datetime, timedelta

import requests
from bs4 import BeautifulSoup
#import html2text
from datetime import datetime



def get_minutes_list(from_date='20000101'): # 기존 20050101 에서 20000101변경 
    prefix_addr = "https://www.bok.or.kr"
    from_date = datetime.strptime(from_date, '%Y%m%d')
    temp =0
    for pageIndex in range(1, 2): ## 46 페이지 == 2000,1,13, 49페이지 까지 있다. 전체 대상 
        url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(pageIndex)
        user_agent = 'Mozilla/5.0'
        headers = {'User-Agent': user_agent}
        page = requests.get(url, headers=headers)

        try:
            soup = BeautifulSoup(page.content, 'html.parser')
            brdList = soup.find_all('span', class_='col m10 s10 x9 ctBx')
            datainfo = soup.find_all('div', class_='col s12 dataInfo')
            print("brdList : ",brdList)
            print("datainfo : ",datainfo)

            for post, data in zip(brdList, datainfo):
                guid = prefix_addr + post.a['href'][:-12] # -12 주의 머지?
                print("이동중입니다. : ",guid)
                temp+=1
                dl_file(guid,temp)


                # 여기서 부터 기존파일 보류건 현 파일에서는 필요없음
                """
                desPage = requests.get(guid)
                desSoup = BeautifulSoup(desPage.content, 'html.parser')
                

                title = post.find('span', class_='titlesub').get_text().strip()

                mdate = title[title.find(')(') + 2:-1]
                if mdate[-1] == '.':
                    mdate = mdate[:-1]
                mdate = datetime.strptime(mdate, '%Y.%m.%d')

                if mdate < from_date: 
                    break

                rdate = data.find('span', class_='date').get_text().strip()
                rdate = datetime.strptime(rdate[3:], '%Y.%m.%d')

                #get_minutes_file(guid, mdate, rdate)
                """

                """
                description = desSoup.find('div', class_='dbData').get_text().strip() #공백 제거 
                print("description : ", description)
                if description.replace(' ', '').find('통화정책방향') >= 0:
                    title = post.find('span', class_='titlesub').get_text().strip()

                    mdate = title[title.find(')(') + 2:-1]
                    if mdate[-1] == '.':
                        mdate = mdate[:-1]
                    mdate = datetime.strptime(mdate, '%Y.%m.%d')

                    if mdate < from_date: # 잘 이해 안감. 끝날짜보다 작으면 그만둬라?
                        break

                    rdate = data.find('span', class_='date').get_text().strip()
                    rdate = datetime.strptime(rdate[3:], '%Y.%m.%d')

                    get_minutes_file(guid, mdate, rdate)
                """
                



        except:
            print("get url.content error and pass page{} it".format(pageIndex))






#이거 기존 다운로드용 현재파일에서는 필요없음
def get_minutes_file(page_addr, mdate, rdate):
    file_header = 'data/minutes/pdf/KO_'
    prefix_addr = "https://www.bok.or.kr" # https

    page = requests.get(page_addr)
    soup = BeautifulSoup(page.content, 'html.parser')

    print("...")
    try:
        links = soup.find('div', class_='addfile').find_all('a')

        for link in links:
            filename = link.get_text()
            filename = filename.replace('\r', '').replace('\t', '').replace('\n', '')
            
            ##실제 <a href="/portal/cmmn/file/fileDown.do?menuNo=200761&amp;atchFileId=FILE_000000000030802&amp;fileSn=1">
						##			금융통화위원회 의사록(2022년도 제7차)(2022.4.14).hwp
						##		</a>
            ## http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&amp;atchFileId=FILE_000000000030802&amp;fileSn=1
            ## 현재 http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000028630&fileSn=2

            if filename[-3:] == 'pdf':
                filename = mdate.strftime('%Y%m%d') + "_" + rdate.strftime('%Y%m%d') + '.pdf'
                file_addr = prefix_addr + link["href"]
                print( "해당 주소로 이동중입니다. ",file_addr)
                file_res = requests.get(file_addr)
                filepath = file_header + filename

                with open(filepath, 'wb') as f:
                    f.write(file_res.content)

                print('save file name : ')
                print(filename)
    except:
        print("get file failed and pass it")

## usl 접속 후 다운로드 하지 

#다운로드용 
def dl_file(url,temp) :
    print("url : ",url)
    user_agent = 'Mozilla/5.0'
    headers = {'User-Agent': user_agent}
    page = requests.get(url, headers=headers)

    print("page.content : ",page.content)
    soup = BeautifulSoup(page.content, 'html.parser')
    file1 = soup.find('div', class_='addfile').find('ul').find('li').find('a').attrs['href']
    name = soup.find('h3', class_='subject').text
    # subject
    print("temp : ",temp," name : ",name)

    link1 = 'http://www.bok.or.kr'+file1

    file_res = requests.get(link1)
    
    with open('{}.pdf'.format(name),'wb') as f :
      f.write(file_res.content)



get_minutes_list(from_date='20000101')


print( "실행이 완료 되었습니당! ")





# 새 섹션

In [ ]:
## usl 접속 후 다운로드 하지 


def dl_file(url) :
    temp = 0
    user_agent = 'Mozilla/5.0'
    headers = {'User-Agent': user_agent}
    page = requests.get(url, headers=headers)

    soup = BeautifulSoup(page.content, 'html.parser')
    file1 = soup.find_all('div', class_='addfile').find('a').attrs['href']
    
    print("temp : ",temp," datainfo : ",file1)
    temp = temp +1
